In [1]:
import lxml
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

In [2]:
from fake_useragent import UserAgent                
ua = UserAgent()
headers = {'User-Agent': ua.random}


In [3]:
url = "https://www.transfermarkt.es/valencia-cf/kader/verein/1049/plus/0/galerie/0?saison_id=2019"

In [4]:
plantilla20 = requests.get(url, headers=headers)


In [5]:
plantilla20 = BeautifulSoup(plantilla20.text, "html.parser")

In [6]:
temp20={"Nombre":[],
        "Posicion":[],
        "Edad":[],
        "Nacionalidad":[],
        "Valor":[]
        }

SACAR EL VALOR DEL JUGADOR EN EL EQUIPO

In [7]:
for valor in plantilla20.find_all("td", class_ = "rechts hauptlink"):
    temp20["Valor"].append(valor.text)

SACAR NOMBRE JUGADOR

In [8]:
nombres = plantilla20.find_all('img', class_='bilderrahmen-fixed lazy lazy')
for nombre in nombres:
    #jugador24 = nombre["title"]
    temp20["Nombre"].append(nombre["title"])


SACAR EDAD JUGADOR

Veo que desde la posicion 1 y cada 4 saco el TD donde esta la edad ya que ese campo no tiene ningún atributo o clase que pueda diferenciar del resto.
Saco la longitud total de el td para saber su longitud y recorrer cada campo de la edad.

In [9]:
edad = plantilla20.find_all("td",class_ = "zentriert")

longitud=(len(edad)-4) #Saco el total y en el range le pongo 97 porque es el último valor de la cadena que queremos sacar.

valores_edad = [edad[i].text.strip() for i in range(1, longitud, 4)]
temp20["Edad"]=valores_edad

SACAR LA POSICION
Como en el anterior no tenemos una etiqueta ni nada que nos indique el campo de la posicion tenemos que sacarlo
a traves de la posicion de la etiqueta.

In [10]:
posiciones = plantilla20.find_all("table",class_ = "inline-table")

longitud_pos = (len(posiciones))
for i in range(longitud_pos):
    #print(posiciones[i].find_all("td")[2].text.strip())
    temp20["Posicion"].append(posiciones[i].find_all("td")[2].text.strip())

SACAR NACIONALIDAD

saco el td con la clase, veo que los que tienen doble nacionalidad ocupan td diferentes por eso le decimos que coja todos los "img" y la primera posicion de cada td.

In [11]:
nacionalidades = plantilla20.find_all("td", class_='zentriert')
for nacion in nacionalidades:
    img = nacion.find_all("img", class_="flaggenrahmen")
    if img:
        #print(img[0].get("title"))
        temp20["Nacionalidad"].append(img[0].get("title"))

In [12]:
temporada20 =  pd.DataFrame(temp20)

In [13]:
temporada20.head(10)

,Nombre,Posicion,Edad,Nacionalidad,Valor
0,Jasper Cillessen,Portero,31,Países Bajos,"14,50 mill. €"
1,Jaume Doménech,Portero,29,España,"6,00 mill. €"
2,Cristian Rivero,Portero,22,España,250 mil €
3,Gabriel Paulista,Defensa central,29,Brasil,"17,50 mill. €"
4,Mouctar Diakhaby,Defensa central,23,Guinea,"17,50 mill. €"
5,Eliaquim Mangala,Defensa central,29,Francia,"6,00 mill. €"
6,Cristiano Piccini,Defensa central,27,Italia,"6,00 mill. €"
7,Ezequiel Garay,Defensa central,33,Argentina,"4,80 mill. €"
8,Javi Jiménez,Defensa central,23,España,250 mil €
9,Guillem Molina,Defensa central,20,España,150 mil €


In [14]:
temporada20.Edad = temporada20.Edad.astype(int)

In [15]:
temporada20.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Nombre        33 non-null     object
 1   Posicion      33 non-null     object
 2   Edad          33 non-null     int32 
 3   Nacionalidad  33 non-null     object
 4   Valor         33 non-null     object
dtypes: int32(1), object(4)
memory usage: 1.3+ KB


In [16]:
temporada20.Valor = temporada20.Valor.str.replace("[^\d,.]", "", regex=True).str.replace(",", ".").str.extract(r'(\d+.\d+|\d+)').astype(float)

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\raul_\AppData\Local\Temp\ipykernel_24208\4089477190.py:1: SyntaxWarning: invalid escape sequence '\d'
  temporada20.Valor = temporada20.Valor.str.replace("[^\d,.]", "", regex=True).str.replace(",", ".").str.extract(r'(\d+.\d+|\d+)').astype(float)


Pasamos los valores a millones, sabemos que los valores altos que habian no corresponden al valor en millones ya que sabemos que 
la plantilla del valencia no tiene jugadores tan valiosos.

In [17]:
temporada20.loc[temporada20.Valor >= 100,"Valor"] = temporada20.loc[temporada20.Valor >= 100,"Valor"]/1000


In [18]:
temporada20["Valor millones"] = temporada20.Valor 

In [19]:
temporada20 = temporada20.drop(columns=["Valor"])

In [20]:
temporada20

,Nombre,Posicion,Edad,Nacionalidad,Valor millones
0,Jasper Cillessen,Portero,31,Países Bajos,14.50
1,Jaume Doménech,Portero,29,España,6.00
2,Cristian Rivero,Portero,22,España,0.25
3,Gabriel Paulista,Defensa central,29,Brasil,17.50
4,Mouctar Diakhaby,Defensa central,23,Guinea,17.50
5,Eliaquim Mangala,Defensa central,29,Francia,6.00
6,Cristiano Piccini,Defensa central,27,Italia,6.00
7,Ezequiel Garay,Defensa central,33,Argentina,4.80
8,Javi Jiménez,Defensa central,23,España,0.25
9,Guillem Molina,Defensa central,20,España,0.15


In [21]:
#temporada22["Valor millones"].fillna(temporada22["Valor millones"].min, inplace=True)
min_valor = temporada20["Valor millones"].min()
temporada20["Valor millones"].fillna(min_valor, inplace=True)

C:\Users\raul_\AppData\Local\Temp\ipykernel_24208\2857528133.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  temporada20["Valor millones"].fillna(min_valor, inplace=True)


In [22]:
ruta="D:\DATASCIENCE\Repositorios\ONLINE_DS_THEBRIDGE_RAULSOLER\Proyecto EDA\LIMGOHOME\DatasLimpios\Temporada20_19.csv"

temporada20.to_csv(ruta, index="False")

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\raul_\AppData\Local\Temp\ipykernel_24208\1832772298.py:1: SyntaxWarning: invalid escape sequence '\D'
  ruta="D:\DATASCIENCE\Repositorios\ONLINE_DS_THEBRIDGE_RAULSOLER\Proyecto EDA\LIMGOHOME\DatasLimpios\Temporada20_19.csv"
